### Run sample code from Original TAACO github
##### source: https://github.com/LCR-ADS-Lab/TAACO

In [1]:
# from TAACOnoGUI import runTAACO
# import os

# #set processing options
# sampleVars = {"sourceKeyOverlap" : False, "sourceLSA" : False, "sourceLDA" : False, "sourceWord2vec" : False, "wordsAll" : True, "wordsContent" : True, "wordsFunction" : True, "wordsNoun" : True, "wordsPronoun" : True, "wordsArgument" : True, "wordsVerb" : True, "wordsAdjective" : True, "wordsAdverb" : True, "overlapSentence" : True, "overlapParagraph" : True, "overlapAdjacent" : True, "overlapAdjacent2" : True, "otherTTR" : True, "otherConnectives" : True, "otherGivenness" : True, "overlapLSA" : True, "overlapLDA" : True, "overlapWord2vec" : True, "overlapSynonym" : True, "overlapNgrams" : True, "outputTagged" : False, "outputDiagnostic" : False}

# # Run TAACO on a folder of texts ("ELLIPSE_Sample/"), give the output file a name ("packageTest.csv), provide output for particular indices/options (as defined in sampleVars)
# taaco_dir = "/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO"
# indir = os.path.join(taaco_dir, "ELLIPSE_Sample")
# runTAACO(indir,"packageTestself.csv",sampleVars)

In [2]:
import pandas as pd
ref_df = pd.read_csv("packageTest.csv")

### Initiate reimplemented TAACO processor

In [3]:
from TAACO_pure_txt import TAACOProcessor
sampleVars = {"sourceKeyOverlap" : False, "sourceLSA" : False, "sourceLDA" : False, "sourceWord2vec" : False, "wordsAll" : True, "wordsContent" : True, "wordsFunction" : True, "wordsNoun" : True, "wordsPronoun" : True, "wordsArgument" : True, "wordsVerb" : True, "wordsAdjective" : True, "wordsAdverb" : True, "overlapSentence" : True, "overlapParagraph" : True, "overlapAdjacent" : True, "overlapAdjacent2" : True, "otherTTR" : True, "otherConnectives" : True, "otherGivenness" : True, "overlapLSA" : True, "overlapLDA" : True, "overlapWord2vec" : True, "overlapSynonym" : True, "overlapNgrams" : True, "outputTagged" : False, "outputDiagnostic" : False}
taaco_processor = TAACOProcessor(sampleVars)

/home/administrator/anaconda3/envs/babylm_old_trl/lib/python3.11/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading Spacy
Loading Spacy Model
Loading LSA vector space...
Loading LDA vector space...
Loading word2vec vector space...


In [4]:
import random
import os
# random sample from sample folder
sample_dir = "/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/ELLIPSE_Sample"
# Get a list of .txt files from the directory
all_txt_files = [f for f in os.listdir(sample_dir) if f.endswith('.txt')]

# Randomly sample 10 files
sampled_files = random.sample(all_txt_files, min(10, len(all_txt_files)))
sampled_files


['2DC55C3EFD7D.txt',
 '2CD73298526F.txt',
 '2C7A6E04F7D0.txt',
 '1B2B5D08014D.txt',
 '0DC7DEF31BE5.txt',
 '0ADBB145FC2C.txt',
 '1BE17E2DA549.txt',
 '0E03218FDD2F.txt',
 '0E5C8FE6867B.txt',
 '2D5A9BEEB30D.txt']

### Test with the 10 sample file

In [5]:
# compare utils
def compare_dict_keys(dict1, dict2):
    keys1 = set(dict1.keys())
    keys2 = set(dict2.keys())

    only_in_dict1 = keys1 - keys2
    only_in_dict2 = keys2 - keys1

    print("Keys only in dict1:")
    for key in sorted(only_in_dict1):
        print(f"  {key}")

    print("\nKeys only in dict2:")
    for key in sorted(only_in_dict2):
        print(f"  {key}")
        
def compare_dict_values(d1, d2, skip_keys=['Filename'], float_tol=1e-6):
    mismatches = []

    for key in d1:
        if key in skip_keys:
            continue
        v1, v2 = d1[key], d2.get(key)

        # Check exact match
        if v1 == v2:
            continue

        # Try float comparison (for rounding differences)
        try:
            f1, f2 = float(v1), float(v2)
            if abs(f1 - f2) <= float_tol:
                continue
        except (ValueError, TypeError):
            pass  # Not comparable as float

        mismatches.append((key, v1, v2))

    # Print mismatched keys
    if mismatches:
        print("Mismatches found:")
        for key, v1, v2 in mismatches:
            print(f"  {key}: dict1 = {v1}, dict2 = {v2}")
    else:
        print("All values match (within tolerance).")

In [8]:
for temp_file_name in sampled_files:
    temp_file_path = os.path.join(sample_dir, temp_file_name)
    with open(temp_file_path, "r", encoding="utf-8") as f:
        content = f.read()
    
    temp_result_dict = taaco_processor.process_text(content)
    row_dict = ref_df[ref_df['Filename'] == temp_file_name].to_dict(orient='records')[0]
    
    print(f"lsa_1_all_sent: dict1 = {row_dict['lsa_1_all_sent']}, dict2 = {temp_result_dict['lsa_1_all_sent']}")
    print(f"lsa_1_all_para: dict1 = {row_dict['lsa_1_all_para']}, dict2 = {temp_result_dict['lsa_1_all_para']}")
    compare_dict_values(row_dict, temp_result_dict)
    print('--------------------------------')

/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: invalid value encountered in divide
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))


lsa_1_all_sent: dict1 = 0.1991887674738411, dict2 = 0.19918876747384115
lsa_1_all_para: dict1 = 0.6407977874685613, dict2 = 0.6407977874685613
Mismatches found:
  lsa_2_all_sent: dict1 = 0.6406531061529056, dict2 = 0.27930461329586465
  lsa_2_all_para: dict1 = 0.8680621193884559, dict2 = 0.761624447880143
  lda_1_all_sent: dict1 = 0.995958793591496, dict2 = 0.9284036333713962
  lda_2_all_sent: dict1 = 0.9987034687003784, dict2 = 0.9652178179355368
  lda_1_all_para: dict1 = 0.9971237670117952, dict2 = 0.992969756202071
  lda_2_all_para: dict1 = 0.9975038151519175, dict2 = 0.9933527419953432
  word2vec_1_all_sent: dict1 = 0.8878489149927214, dict2 = 0.4144912908109536
  word2vec_2_all_sent: dict1 = 0.917700934786676, dict2 = 0.5127677884195543
  word2vec_1_all_para: dict1 = 0.9268551047179744, dict2 = 0.7736774238211998
  word2vec_2_all_para: dict1 = 0.9233350544626064, dict2 = 0.8134323191650548
--------------------------------


/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: invalid value encountered in divide
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))


lsa_1_all_sent: dict1 = 0.3844599803755498, dict2 = 0.3844599803755498
lsa_1_all_para: dict1 = 0.5216777301524778, dict2 = 0.5216777301524778
Mismatches found:
  lsa_2_all_sent: dict1 = 0.7570728844179143, dict2 = 0.45694803334016254
  lsa_2_all_para: dict1 = 0.8098221534850519, dict2 = 0.6348930101882414
  lda_1_all_sent: dict1 = 0.9823857836364108, dict2 = 0.9389752994119284
  lda_2_all_sent: dict1 = 0.9959615615899864, dict2 = 0.9422508613053786
  lda_1_all_para: dict1 = 0.9965941740615266, dict2 = 0.9938505011803672
  lda_2_all_para: dict1 = 0.998226213415732, dict2 = 0.9927654232995149
  word2vec_1_all_sent: dict1 = 0.9045152875123048, dict2 = 0.4840096265203779
  word2vec_2_all_sent: dict1 = 0.9333193262178868, dict2 = 0.5644643880616086
  word2vec_1_all_para: dict1 = 0.9133303790577968, dict2 = 0.7915589201060665
  word2vec_2_all_para: dict1 = 0.9213195025352294, dict2 = 0.8332305595542056
--------------------------------


/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: invalid value encountered in divide
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))


...

/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: invalid value encountered in divide
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))


lsa_1_all_sent: dict1 = 0.6194499911214294, dict2 = 0.6194499911214294
lsa_1_all_para: dict1 = 0.9150092528530218, dict2 = 0.9150092528530218
Mismatches found:
  lsa_2_all_sent: dict1 = 0.8492461738898233, dict2 = 0.6545573425603077
  lsa_2_all_para: dict1 = 0.9491101559343424, dict2 = 0.9165481950846418
  lda_1_all_sent: dict1 = 0.9903297312750436, dict2 = 0.9690152656589064
  lda_2_all_sent: dict1 = 0.99621460239969, dict2 = 0.9732620363875301
  lda_1_all_para: dict1 = 0.9990412963886216, dict2 = 0.9980503066290215
  lda_2_all_para: dict1 = 0.9987747848777944, dict2 = 0.9980587567777622
  word2vec_1_all_sent: dict1 = 0.916507012980274, dict2 = 0.6188460018897609
  word2vec_2_all_sent: dict1 = 0.9401310870623096, dict2 = 0.6622050859015142
  word2vec_1_all_para: dict1 = 0.9357054675316944, dict2 = 0.8526240765001416
  word2vec_2_all_para: dict1 = 0.9332666078478432, dict2 = 0.8448184233107444
--------------------------------


/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: invalid value encountered in divide
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))
/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: divide by zero encountered in log2
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))
/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: invalid value encountered in multiply
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))


lsa_1_all_sent: dict1 = 0.5030133607122181, dict2 = 0.5030133607122181
lsa_1_all_para: dict1 = 0.7736192844161436, dict2 = 0.7736192844161436
Mismatches found:
  lsa_2_all_sent: dict1 = 0.8043589819778953, dict2 = 0.6045151416799414
  lsa_2_all_para: dict1 = 0.9230370152649724, dict2 = 0.8389935640086025
  lda_1_all_sent: dict1 = 0.9826500582952048, dict2 = 0.9433653688357955
  lda_2_all_sent: dict1 = 0.9897568896291552, dict2 = 0.9626037145181142
  lda_1_all_para: dict1 = 0.9992727614921038, dict2 = 0.9989492873729637
  lda_2_all_para: dict1 = 0.9991062270502968, dict2 = 0.9986662078726382
  word2vec_1_all_sent: dict1 = 0.8715396409237917, dict2 = 0.4627433237680777
  word2vec_2_all_sent: dict1 = 0.9104930262736066, dict2 = 0.5833224847715293
  word2vec_1_all_para: dict1 = 0.9455935591352947, dict2 = 0.8279309492306016
  word2vec_2_all_para: dict1 = 0.947572443928341, dict2 = 0.8772072285088234
--------------------------------
lsa_1_all_sent: dict1 = 0.6786058604865789, dict2 = 0.6786

/home/administrator/projects/babylm_reformed/babylm-interaction/TnD_reformed/experiments/formal_experiment1/TAACO/TAACO_pure_txt.py:788: RuntimeWarning: invalid value encountered in divide
  return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))


### reproduce from original function

0. lsa1_all_sent is average similarity of sent1 against sent2, lsa2_all_sent is sent1 against sent2 + sent3; if the bug happens, sent2 will be sent2 + sent3, sent3 will be sent3 + sent4, with that the following compare will be sent2 + sent3 vs sent3 + sent4, ofc similarity increasee
1. increase amount from lsa_1_all_sent to lsa_2_all_sent is too large for sample dict from original code
2. the following metrics converge to 1 in order to the sequence in implementation, suggesting after eacch call with seg = 2
3. following code


In [9]:
import platform
import numpy as np
# from collections import Counter
from operator import itemgetter
import glob
import math
from collections import Counter
import math

def safe_divide(numerator, denominator):
    if denominator == 0:
        index = 0
    else: index = numerator/denominator
    return(index)


def lda_divergence(text_list_1,text_list_2,dict,nvectors = 300):

    def vector_av(text_list):
        n_items = 0
        l = []
        for i in range(nvectors):
            l.append(0)

        for items in text_list:
            if items not in dict:
                continue
            else:
                n_columns = 0
                n_items+=1
                for vector in dict[items]:
                    l[n_columns] += float(vector)
                    n_columns +=1
        try:
            for x in range(len(l)): #normalize for number of words
                l[x] = safe_divide(l[x],n_items)
            
            div = np.linalg.norm(l, ord = 1) #linear algebra normalization - all items in list sum to 1
            for x in range(len(l)):
                l[x] = safe_divide(x,div)

        except ZeroDivisionError:
            l = "null"
        
        return(l)

    def jsdiv(P, Q):
        """Compute the Jensen-Shannon divergence between two probability distributions.

        Input
        -----
        P, Q : array-like
            Probability distributions of equal length that sum to 1
        """

        def _kldiv(A, B):
            return(np.sum([v for v in A * np.log2(A/B) if not np.isnan(v)]))

        P = np.array(P)
        Q = np.array(Q)

        M = 0.5 * (P + Q)

        return(0.5 * (_kldiv(P, M) +_kldiv(Q, M)))


    list1 = vector_av(text_list_1)
    list2 = vector_av(text_list_2)
    #print list1[:20]
    if list1 == "null" or list2 == "null":
        divergence = "null"
    else:
        divergence = 1 - jsdiv(list1,list2)
        if divergence >= 0: #this silliness controls for some wacky output. This was in Mihai's code... not sure how the wackiness occurs (it is rare)
            divergence = divergence
        else:
            divergence = "null"
    return(divergence)

def lsa_similarity(text_list_1,text_list_2,lsa_matrix_dict,index_list = None,index_name= None,header_list = None,lsa_type = "fwd",nvectors = 300):

    def vector_av(text_list):
        n_items = 0
        l = []
        for i in range(nvectors):
            l.append(0)

        for items in text_list:
            if items not in lsa_matrix_dict:
                continue
            else:
                n_columns = 0
                n_items+=1
                for vector in lsa_matrix_dict[items]:
                    l[n_columns] += float(vector)
                    n_columns +=1

        #n_columns = 0
        #for items in l:
        #	l[n_columns] = l[n_columns]/n_items

        sum_count = 0
        for items in l:
            sum_count += math.pow(items,2)
        sqrt_sum = math.sqrt(sum_count)
    

        return([l, sqrt_sum])

    list1 = vector_av(text_list_1)
    list2 = vector_av(text_list_2)

    try:
        sum_count_2 = 0
        for items in range(len(list1[0])):
            sum_count_2+= (list1[0][items]*list2[0][items])


        cosine_sim = sum_count_2/(list1[1]*list2[1])

    except ZeroDivisionError:
        cosine_sim = "null"

    if header_list is not None:
        header_list.append(index_name)
        index_list.append(cosine_sim)

    if header_list is None:
        #print cosine_sim
        return(cosine_sim)

def segment_compare(header_list,index_list,text_list_of_lists, seg, name_suffix, lsa_matrix_dict,type = "lsa"): 

    #print list
    ## need to add check to ensure that list is a list of lists
    
    #first we have counters:
    
    n_segments = len(text_list_of_lists) #number of sentences or paragraphs
    
    #this next section deals with texts that only have one segment
    if n_segments < (seg+1):
        if seg == 1:
            header_list.append(name_suffix)
            index_list.append(0)
                                        
        if seg == 2:
            header_list.append(name_suffix)
            index_list.append(0)

    #this is the "normal" procedure
    else:	
                        
        denominator = 0
        counter = 0

        for number in range(n_segments-seg):
        
            item_list = text_list_of_lists[number] #e.g., sentence 1
            comparison_list = text_list_of_lists[number + 1]#e.g., sentence 2
            if seg == 2:
                for items in text_list_of_lists[number +2]:
                    comparison_list.append(items)
            
            #print "item list:", item_list
            #print "comparison_list:", comparison_list
            
            if type == "lsa":
                index = lsa_similarity(item_list,comparison_list,lsa_matrix_dict)
                if index == "null":
                    continue
                else:
                    counter += index		
                    denominator +=1

            if type == "lda":
                index = lda_divergence(item_list,comparison_list,lsa_matrix_dict)
                
                if index == "null":
                    continue
                else:
                    counter += index		
                    denominator +=1
        
        index_list.append(safe_divide(counter,denominator))		
        header_list.append(name_suffix)

In [13]:
temp_header_list = []
temp_index_list = [] 
sent_lemma_list = [['a', 'program', 'that', 'benefit', 'both', 'old', 'and', 'younger', 'students'], ['"have', 'you', 'ever', 'experience', 'a', 'program', 'that', 'involve', 'both', 'old', 'and', 'young', 'student', 'or', 'would', 'the', 'student', 'be', 'able', 'to', 'admire', 'each', 'other', '"'], ['old', 'student', 'have', 'more', 'experience', 'than', 'the', 'young', 'student', 'which', 'make', 'it', 'easy', 'for', 'they', 'to', 'handle', 'problem'], ['it', 'also', 'help', 'they', 'to', 'build', 'a', 'strong', 'relationship', 'since', 'they', 'both', 'come', 'from', 'different', 'background'], ['it', 'will', 'be', 'easy', 'for', 'they', 'to', 'bring', 'up', 'idea', 'for', 'a', 'particular', 'project', 'because', 'they', 'both', 'have', 'different', 'experience'], ['the', 'program', 'will', 'help', 'both', 'group', 'of', 'student', 'because', 'they', 'can', 'have', 'thing', 'like', 'competition', 'and', 'also', 'they', 'can', 'both', 'learn', 'from', 'each', 'other', 'and', 'also', 'achieve', 'the', 'goal', 'they', 'have', 'plan', 'for', 'that', 'class', 'in', 'particularly', 'if', 'they', 'work', 'as', 'a', 'team'], ['firstly', 'such', 'a', 'program', 'will', 'benefit', 'both', 'old', 'and', 'young', 'student', 'because', 'they', 'can', 'have', 'thing', 'such', 'as', 'competition'], ['both', 'student', 'will', 'try', 'their', 'good', 'and', 'put', 'in', 'much', 'effort', 'because', 'either', 'they', 'want', 'to', 'be', 'ahead', 'of', 'each', 'other', 'or', 'they', 'want', 'to', 'succeed', 'in', 'that', 'program'], ['they', 'make', 'sure', 'that', 'they', 'value', 'their', 'time', 'and', 'make', 'their', 'book', 'their', '"', 'good', 'friend', '"'], ['there', 'be', 'a', 'quote', 'that', 'state', 'that', '"', 'education', 'be', 'the', 'key', 'to', 'success', '"'], ['when', 'student', 'try', 'to', 'compete', 'with', 'other', 'they', 'learn', 'hard', 'to', 'attain', 'their', 'dream', 'and', 'also', 'to', 'make', 'people', 'believe', 'in', 'they'], ['such', 'competition', 'make', 'student', 'learn', 'hard', 'and', 'also', 'make', 'they', 'successful'], ['secondly', 'such', 'a', 'program', 'will', 'benefit', 'both', 'student', 'because', '"', 'they', 'can', 'both', 'learn', 'from', 'each', 'other', '"'], ['for', 'instance', 'if', 'i', 'have', 'be', 'pair', 'with', 'someone', 'who', 'be', 'three', 'year', 'old', 'than', 'i', 'to', 'do', 'an', 'assignment', 'it', 'make', 'it', 'easy', 'for', 'we', 'to', 'complete', 'the', 'assignment'], ['it', 'be', 'because', 'we', 'be', 'both', 'from', 'different', 'background', 'and', 'also', 'raise', 'up', 'and', 'train', 'by', 'different', 'people'], ['that', 'alone', 'will', 'help', 'we', 'learn', 'from', 'each', 'other', 'because', 'we', 'will', 'not', 'like', 'to', 'work', 'with', 'each', 'other', 'without', 'get', 'to', 'know', 'ourselves'], ['when', 'we', 'realize', 'we', 'be', 'from', 'different', 'place', 'we', 'will', 'be', 'interested', 'in', 'know', 'a', 'little', 'bit', 'about', 'each', 'other', "'s", 'background', 'and', 'how', 'we', 'do', 'our', 'thing'], ['learn', 'from', 'each', 'other', 'benefit', 'both', 'of', 'we', 'especially', 'when', 'we', 'know', 'about', 'our', 'like', 'and', 'dislike'], ['lastly', 'both', 'student', 'benefit', 'from', 'such', 'a', 'program', 'because', 'they', 'can', 'achieve', 'the', 'goal', 'they', 'have', 'plan', 'for', 'that', 'class', 'particularly', 'if', 'they', 'work', 'as', 'a', 'team'], ['there', 'be', 'a', 'quote', 'i', 'remember', 'that', 'say', 'that', '"', 'two', 'head', 'be', 'well', 'than', 'one', '"'], ['for', 'example', 'if', 'one', 'person', 'will', 'be', 'able', 'to', 'carry', 'twenty', 'out', 'of', 'fifty', 'book', 'then', 'two', 'people', 'will', 'be', 'able', 'to', 'carry', 'all', 'the', 'fifty', 'book', 'or', 'even', 'more'], ['the', 'student', 'have', 'goal', 'and', 'dream', 'they', 'want', 'to', 'acquire', 'in', 'life'], ['however', 'since', 'they', 'both', 'have', 'the', 'same', 'purpose', 'they', 'will', 'decide', 'to', 'work', 'together', 'to', 'help', 'each', 'other', 'reach', 'their', 'goal'], ['conclusively', 'both', 'student', 'will', 'benefit', 'from', 'such', 'a', 'program', 'because'], ['they', 'can', 'compete', 'against', 'each', 'other', 'and', 'put', 'in', 'much', 'effort', 'to', 'reach', 'their', 'dream', 'to', 'also', 'learn', 'about', 'each', 'other', 'and', 'to', 'help', 'each', 'other', 'strive', 'for', 'their', 'goal'], ['the', 'effect', 'of', 'a', 'program', 'that', 'help', 'both', 'old', 'and', 'young', 'student', 'to', 'pair', 'up', 'and', 'help', 'each', 'other', 'have', 'be', 'list', 'in', 'the', 'essay', 'above'], ['student', 'can', 'be', 'assist', 'by', 'people', 'who', 'be', 'old', 'young', 'or', 'even', 'of', 'a', 'different', 'race', 'or', 'gender']]
original_sent_lemma_list = [['a', 'program', 'that', 'benefit', 'both', 'old', 'and', 'younger', 'students'], ['"have', 'you', 'ever', 'experience', 'a', 'program', 'that', 'involve', 'both', 'old', 'and', 'young', 'student', 'or', 'would', 'the', 'student', 'be', 'able', 'to', 'admire', 'each', 'other', '"'], ['old', 'student', 'have', 'more', 'experience', 'than', 'the', 'young', 'student', 'which', 'make', 'it', 'easy', 'for', 'they', 'to', 'handle', 'problem'], ['it', 'also', 'help', 'they', 'to', 'build', 'a', 'strong', 'relationship', 'since', 'they', 'both', 'come', 'from', 'different', 'background'], ['it', 'will', 'be', 'easy', 'for', 'they', 'to', 'bring', 'up', 'idea', 'for', 'a', 'particular', 'project', 'because', 'they', 'both', 'have', 'different', 'experience'], ['the', 'program', 'will', 'help', 'both', 'group', 'of', 'student', 'because', 'they', 'can', 'have', 'thing', 'like', 'competition', 'and', 'also', 'they', 'can', 'both', 'learn', 'from', 'each', 'other', 'and', 'also', 'achieve', 'the', 'goal', 'they', 'have', 'plan', 'for', 'that', 'class', 'in', 'particularly', 'if', 'they', 'work', 'as', 'a', 'team'], ['firstly', 'such', 'a', 'program', 'will', 'benefit', 'both', 'old', 'and', 'young', 'student', 'because', 'they', 'can', 'have', 'thing', 'such', 'as', 'competition'], ['both', 'student', 'will', 'try', 'their', 'good', 'and', 'put', 'in', 'much', 'effort', 'because', 'either', 'they', 'want', 'to', 'be', 'ahead', 'of', 'each', 'other', 'or', 'they', 'want', 'to', 'succeed', 'in', 'that', 'program'], ['they', 'make', 'sure', 'that', 'they', 'value', 'their', 'time', 'and', 'make', 'their', 'book', 'their', '"', 'good', 'friend', '"'], ['there', 'be', 'a', 'quote', 'that', 'state', 'that', '"', 'education', 'be', 'the', 'key', 'to', 'success', '"'], ['when', 'student', 'try', 'to', 'compete', 'with', 'other', 'they', 'learn', 'hard', 'to', 'attain', 'their', 'dream', 'and', 'also', 'to', 'make', 'people', 'believe', 'in', 'they'], ['such', 'competition', 'make', 'student', 'learn', 'hard', 'and', 'also', 'make', 'they', 'successful'], ['secondly', 'such', 'a', 'program', 'will', 'benefit', 'both', 'student', 'because', '"', 'they', 'can', 'both', 'learn', 'from', 'each', 'other', '"'], ['for', 'instance', 'if', 'i', 'have', 'be', 'pair', 'with', 'someone', 'who', 'be', 'three', 'year', 'old', 'than', 'i', 'to', 'do', 'an', 'assignment', 'it', 'make', 'it', 'easy', 'for', 'we', 'to', 'complete', 'the', 'assignment'], ['it', 'be', 'because', 'we', 'be', 'both', 'from', 'different', 'background', 'and', 'also', 'raise', 'up', 'and', 'train', 'by', 'different', 'people'], ['that', 'alone', 'will', 'help', 'we', 'learn', 'from', 'each', 'other', 'because', 'we', 'will', 'not', 'like', 'to', 'work', 'with', 'each', 'other', 'without', 'get', 'to', 'know', 'ourselves'], ['when', 'we', 'realize', 'we', 'be', 'from', 'different', 'place', 'we', 'will', 'be', 'interested', 'in', 'know', 'a', 'little', 'bit', 'about', 'each', 'other', "'s", 'background', 'and', 'how', 'we', 'do', 'our', 'thing'], ['learn', 'from', 'each', 'other', 'benefit', 'both', 'of', 'we', 'especially', 'when', 'we', 'know', 'about', 'our', 'like', 'and', 'dislike'], ['lastly', 'both', 'student', 'benefit', 'from', 'such', 'a', 'program', 'because', 'they', 'can', 'achieve', 'the', 'goal', 'they', 'have', 'plan', 'for', 'that', 'class', 'particularly', 'if', 'they', 'work', 'as', 'a', 'team'], ['there', 'be', 'a', 'quote', 'i', 'remember', 'that', 'say', 'that', '"', 'two', 'head', 'be', 'well', 'than', 'one', '"'], ['for', 'example', 'if', 'one', 'person', 'will', 'be', 'able', 'to', 'carry', 'twenty', 'out', 'of', 'fifty', 'book', 'then', 'two', 'people', 'will', 'be', 'able', 'to', 'carry', 'all', 'the', 'fifty', 'book', 'or', 'even', 'more'], ['the', 'student', 'have', 'goal', 'and', 'dream', 'they', 'want', 'to', 'acquire', 'in', 'life'], ['however', 'since', 'they', 'both', 'have', 'the', 'same', 'purpose', 'they', 'will', 'decide', 'to', 'work', 'together', 'to', 'help', 'each', 'other', 'reach', 'their', 'goal'], ['conclusively', 'both', 'student', 'will', 'benefit', 'from', 'such', 'a', 'program', 'because'], ['they', 'can', 'compete', 'against', 'each', 'other', 'and', 'put', 'in', 'much', 'effort', 'to', 'reach', 'their', 'dream', 'to', 'also', 'learn', 'about', 'each', 'other', 'and', 'to', 'help', 'each', 'other', 'strive', 'for', 'their', 'goal'], ['the', 'effect', 'of', 'a', 'program', 'that', 'help', 'both', 'old', 'and', 'young', 'student', 'to', 'pair', 'up', 'and', 'help', 'each', 'other', 'have', 'be', 'list', 'in', 'the', 'essay', 'above'], ['student', 'can', 'be', 'assist', 'by', 'people', 'who', 'be', 'old', 'young', 'or', 'even', 'of', 'a', 'different', 'race', 'or', 'gender']]

segment_compare(temp_header_list,temp_index_list,sent_lemma_list, 2, "lsa_2_all_sent", taaco_processor.lsa_dict)

In [14]:
original_sent_lemma_list == sent_lemma_list

False

In [15]:
original_sent_lemma_list[0] == sent_lemma_list[0]

True

In [16]:
original_sent_lemma_list[1] == sent_lemma_list[1]

False

In [18]:
original_sent_lemma_list[3] + original_sent_lemma_list[4] == sent_lemma_list[3]

True